# Project 3: Reddit Scrape and NLP
## Omar Raheem, Data Scientist 

### Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import time
import requests
import json
pd.set_option("display.max_colwidth", 1)
import regex as re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer ,TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
import nltk
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from sklearn.naive_bayes import MultinomialNB
pd.set_option('display.max_columns', 500)

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

### Reddit Scrape

In [3]:
####I'm pulling all my comments at the top because I don't want to run this scrape again

#function to scrape reddit subreddit
#This function will scrappe submission, skip every 30 post, scrape 100 post at a time
#Give back the title, body, name of subreddit, when it was post, the username, number of comments, number of upvotes and tells 
#if its an original post


# def query_pushshift(subreddit, kind='submission', skip=30, times=100, 
#                     subfield = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self'],
#                     comfields = ['body', 'score', 'created_utc']):
    
#     stem = "https://api.pushshift.io/reddit/search/{}/?subreddit={}&size=500".format(kind, subreddit)
#     mylist = []
    
#     for x in range(1, times):
        
#         URL = "{}&after={}d".format(stem, skip * x)
#         print(URL)
#         response = requests.get(URL)
#         assert response.status_code == 200
#         mine = response.json()['data']
#         df = pd.DataFrame.from_dict(mine)
#         mylist.append(df)
#         time.sleep(2)
        
#     full = pd.concat(mylist, sort=False)
    
#     if kind == "submission":
        
#         full = full[subfield]
        
#         full = full.drop_duplicates()
        
#         full = full.loc[full['is_self'] == True]
        
#     def get_date(created):
#         return dt.date.fromtimestamp(created)
    
#     _timestamp = full["created_utc"].apply(get_date)
    
#     full['timestamp'] = _timestamp

#     print(full.shape)
    
#     return full 

In [4]:
# 
venbros = pd.read_csv('./venbros.csv')

In [5]:
venbros.head()

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
0,"Don't know where else to ask, looking for James Urbaniak clip.","It wasn't Venture Bros related, and I don't remember where I saw it, all I remember is James saying ""My dad hit a deer once.... it was a bad scene..."" and the vibe of the video got super dark. Please help me.",venturebros,1551678427,Formaldehyde_N_Seek,2,5,True,2019-03-04
1,I want to hear everyone's theories about that scary bear,He played kind of a big role in season Seven andb he showed up in a lot of background shots who is je what does he want what does every one think my favourite part of venture bros is the great Dan theroux about what's going to happen.,venturebros,1551709250,Jumbleduplya,12,11,True,2019-03-04
2,Why isn’t S7 part of the [as] app marathon or Hulu?,"I’ve had the marathon on in the background occasionally the past few days, and I noticed no new episodes are a part of the 24/7 marathon. The marathon currently ends on “Red Means Stop” and immediately goes to “The Terrible Secret of Turtle Bay,” same as it has since debuting on the app.\n\nS7 premiered 7 months ago and wrapped up five months ago. There are zero episodes available on Hulu, despite 1-6 + specials (except From the Ladle to the Grave) being there, and the [as] app itself only offers S7E3-S7E6, a mere four episodes. \n\nWhy is S7 unavailable in its entirety without buying the season on Amazon? Who wants to own the episodes without commentary from DH and JP? I found it kinda odd, especially considering new episodes of most other shows seem to get added to their respective [as] marathons as they premiere and otherwise would be available on Hulu, including most of [as]’s other 2018 offerings. Not to mention, it’s not like there’s a physical release of the season available yet",venturebros,1551746928,toftr,4,7,True,2019-03-04
3,"Some thoughts about if ""everything has a soul"" conversation between henchmen.","I had an odd connect the dots style thought chain that lead me to venture brothers, as happens often enough. This time it was about 24 telling 21 that everything has a soul, but babies don't get a soul until they are one. As well as saying ""they are not quite souls, but that is the general idea"".\n\nIf plants have this pseudo soul, but babies don't, what if the ""soul"" is a being will to survive? I'll probably get some hot ELI5 style ""you are wrong and here is why"" in the comments, but consider this. \n\nPlants adapt, overcome, and adjust, despite not having any ability to think. They know how to get what they need, and will adapt in crazy ways to get it. We have carnivorous plants for crying out loud!\n\nNow, why don't babies have souls? Human babies don't really have much will to survive, or instincts. They really are basically a ""little crying pooing monster blob"" until they age enough. \n\nMany animals have instincts of some sort when they are born, buuut many others do not. Soul vs no soul. \n\n",venturebros,1551797997,gwarsh41,8,12,True,2019-03-05
4,On the Monarch and Dr. Venture (w/ Spoilers) Theory,"I apologize if this has been covered/theorized before.\n\nDean, Rusty, and the Monarch are all clones of the same person. The original was a milkshake baby like in the movie “Twins” with Arnold and Danny Devito. The difference was that Devito’s character (Rusty) absorbed Arnie (JJ) in vitro. \n\nThe problem with this theory is that there wasn’t a JJ tumor in the Monarch. It could have been a side effect of the clone technology like the Monarch’s eyebrows (abnormal hair grow versus conjoined twin). This could be addressed if the Monarch is a later generation clone than the current Rusty. Jonas could have cloned the Monarch post plane crash thinking Venturion was going to retain his memories post bot makeover.\n\nI still haven’t figured out how Hank fits in. Also, the Helper bot that gave rise the this theory was red (a red herring-bot).",venturebros,1551857665,Nitemare9999,7,5,True,2019-0

In [6]:
usadad = pd.read_csv('./usadad.csv')

usadad.columns

Index(['title', 'selftext', 'subreddit', 'created_utc', 'author',
       'num_comments', 'score', 'is_self', 'timestamp'],
      dtype='object')

In [7]:
#venbros = query_pushshift('venturebros')


In [8]:
#venbros.to_csv(r'./venbros.csv', index=False)

In [9]:
#usadad = query_pushshift('americandad')

In [10]:
#usadad.to_csv(r'./usadad.csv', index=False)

In [11]:
#venbros.selftext.iloc[[0]]

### EDA

In [12]:
#rename columns
venbros = venbros.rename(columns={'title': 'title','selftext': 'body'}
                        )

In [13]:
#drop unneeded functions
ven_features = ['title', 'body', 'subreddit']

In [14]:
#map over subreddit column and replace with 1
venbros['subreddit'] = venbros['subreddit'].map({'venturebros': 1})

In [15]:
ven_X = venbros[ven_features]

In [16]:
ven_X.head()

,title,body,subreddit
0,"Don't know where else to ask, looking for James Urbaniak clip.","It wasn't Venture Bros related, and I don't remember where I saw it, all I remember is James saying ""My dad hit a deer once.... it was a bad scene..."" and the vibe of the video got super dark. Please help me.",1
1,I want to hear everyone's theories about that scary bear,He played kind of a big role in season Seven andb he showed up in a lot of background shots who is je what does he want what does every one think my favourite part of venture bros is the great Dan theroux about what's going to happen.,1
2,Why isn’t S7 part of the [as] app marathon or Hulu?,"I’ve had the marathon on in the background occasionally the past few days, and I noticed no new episodes are a part of the 24/7 marathon. The marathon currently ends on “Red Means Stop” and immediately goes to “The Terrible Secret of Turtle Bay,” same as it has since debuting on the app.\n\nS7 premiered 7 months ago and wrapped up five months ago. There are zero episodes available on Hulu, despite 1-6 + specials (except From the Ladle to the Grave) being there, and the [as] app itself only offers S7E3-S7E6, a mere four episodes. \n\nWhy is S7 unavailable in its entirety without buying the season on Amazon? Who wants to own the episodes without commentary from DH and JP? I found it kinda odd, especially considering new episodes of most other shows seem to get added to their respective [as] marathons as they premiere and otherwise would be available on Hulu, including most of [as]’s other 2018 offerings. Not to mention, it’s not like there’s a physical release of the season available yet",1
3,"Some thoughts about if ""everything has a soul"" conversation between henchmen.","I had an odd connect the dots style thought chain that lead me to venture brothers, as happens often enough. This time it was about 24 telling 21 that everything has a soul, but babies don't get a soul until they are one. As well as saying ""they are not quite souls, but that is the general idea"".\n\nIf plants have this pseudo soul, but babies don't, what if the ""soul"" is a being will to survive? I'll probably get some hot ELI5 style ""you are wrong and here is why"" in the comments, but consider this. \n\nPlants adapt, overcome, and adjust, despite not having any ability to think. They know how to get what they need, and will adapt in crazy ways to get it. We have carnivorous plants for crying out loud!\n\nNow, why don't babies have souls? Human babies don't really have much will to survive, or instincts. They really are basically a ""little crying pooing monster blob"" until they age enough. \n\nMany animals have instincts of some sort when they are born, buuut many others do not. Soul vs no soul. \n\n",1
4,On the Monarch and Dr. Venture (w/ Spoilers) Theory,"I apologize if this has been covered/theorized before.\n\nDean, Rusty, and the Monarch are all clones of the same person. The original was a milkshake baby like in the movie “Twins” with Arnold and Danny Devito. The difference was that Devito’s character (Rusty) absorbed Arnie (JJ) in vitro. \n\nThe problem with this theory is that there wasn’t a JJ tumor in the Monarch. It could have been a side effect of the clone technology like the Monarch’s eyebrows (abnormal hair grow versus conjoined twin). This could be addressed if the Monarch is a later generation clone than the current Rusty. Jonas could have cloned the Monarch post plane crash thinking Venturion was going to retain his memories post bot makeover.\n\nI still haven’t figured out how Hank fits in. Also, the Helper bot that gave rise the this theory was red (a red herring-bot).",1


In [17]:
#rename columns
usadad =usadad.rename(columns={'title': 'title','selftext': 'body'})

In [18]:
#drop unneeded functions
usa_features = ['title', 'body', 'subreddit']

In [19]:
#map over subreddit column and replace with 0
usadad['subreddit'] = usadad['subreddit'].map({'americandad': 0})

In [20]:
usa_X = usadad[usa_features]

In [21]:
#check dataframe
usa_X.head()

,title,body,subreddit
0,What songs would you love to her Steve harmonize?,NaN,0
1,What songs would you love to hear Steve harmonize?,NaN,0
2,Stan changing???,"The new episodes that just came out show stan with a somewhat changed personality. Usually character like stan, peter from family guy and homer from the Simpsons are static, they always behave the same as if all the life lessons learned from the previous episodes were nonexistent but stan seems to have changed from a narcissistic douchebag thats self absorbed to a caring yet somehow childish character who still has his own agenda. He cared for roger, he cared sk much he offered to do his roles. He cared for jeff to get a job. So weird how he suddenly chabged, not complaining its just odd and hard to get used to after so many seasons of asshole stan we now got the actual dad stan",0
3,vincewasmolested.com,"Anyone remember this? This was teased in one of the episodes and the page just keeps. crashing, is he website real?",0
4,Whats your favorite episode and why?,I will watch every suggestion posted here.,0


In [22]:
#check shape
usa_X['title'].shape

(4145,)

In [23]:
ven_X['title'].shape

(4450,)

In [24]:
#find nulls in column
ven_X[ven_X['body'].isnull()]

,title,body,subreddit
26,what's your favorite vb quote?,NaN,1
47,What was the evilest act ever committed in the VB universe?,NaN,1
53,Action!,NaN,1
54,Your best Venture Bros inspired username?,NaN,1
58,There was even talk of French toast,NaN,1
111,"If Phantom Limb has an invisible penis, how did he have a dick measuring contest with Hunter Gathers?",NaN,1
125,Sence there's soo much lore. Whats your favorite piece of venture bros law?,NaN,1
126,"Since the venture bros has so much lore, it can be very overwhelmimg for first timers. Try to do your best to explain this madness to someone who hasn't got an hank of an idea.",NaN,1
137,Anywhere I can find venture bros merch like a shirt or something ?,NaN,1
141,I can’t wait for next season because it will be the end of posts about the Monarch butterfly population.,NaN,1


In [25]:
usa_X[usa_X['body'].isnull()]

,title,body,subreddit
0,What songs would you love to her Steve harmonize?,NaN,0
1,What songs would you love to hear Steve harmonize?,NaN,0
18,He HAD to do it!,NaN,0
19,I’m thinking of some joke where a character told a REALLY sad/ disturbing story and someone else responded with “that’s the saddest thing ever!” sounding genuinely upset.,NaN,0
20,Anyone got the lyrics to Old Ulysses?,NaN,0
21,"Okay, it's May 14, 1981. President Jimmy Carter leaves the White House in disgrace. Pope John Paul II is shot in Rome by a deranged Jimmy Carter. Jimmy Carter kidnaps and devours thirteen Christian babies. Hmm. Maybe I should have read a real history book instead of the one Dad wrote.",NaN,0
41,I still wanna meet Maureen Jacket,NaN,0
44,"What season/episode is the scene where that guy David moves into the neighborhood and he says something like “you’ve struck my garage, i feel as though we will not get along.”",NaN,0
53,NYAH!,NaN,0
75,What character of Rogers is your favorite?,NaN,0


In [26]:
#drop nulls
ven_X.dropna(how='any', inplace=True)

In [27]:
usa_X.dropna(how='any', inplace=True)

In [28]:
#drop body. Only going to use data from titles
usa_title= ['title', 'subreddit']

In [29]:
usa_title = usa_X[usa_title]

In [30]:
#check dataframe
usa_title.head()

,title,subreddit
2,Stan changing???,0
3,vincewasmolested.com,0
4,Whats your favorite episode and why?,0
5,"I am Stan Smith, discoverer of Ollie North's gold. You probably know me from the movie they made about my life, starring Bruce Campbell. It was the role that finally got him out of those crappy B movies and into the arms of his husband, Campbell Scott. Who is now Campbell Campbell.",0
6,You go. I'm just gonna stare at the lake and think how I almost killed a baby.,0


In [31]:
features = ['title', 'subreddit']

In [32]:
ven_title = ven_X[features]

In [33]:
ven_title.head()

,title,subreddit
0,"Don't know where else to ask, looking for James Urbaniak clip.",1
1,I want to hear everyone's theories about that scary bear,1
2,Why isn’t S7 part of the [as] app marathon or Hulu?,1
3,"Some thoughts about if ""everything has a soul"" conversation between henchmen.",1
4,On the Monarch and Dr. Venture (w/ Spoilers) Theory,1


In [34]:
#concat the 2 dataframes together from the top to bottom
titles = pd.concat([ven_title, usa_title], axis=0)

In [35]:
#check dataframe
titles.tail()

,title,subreddit
4140,Chocodiles,0
4141,Anybody notice how Francine was an idiot in one episode of American Dad?,0
4142,Why aren't there more people in this group?!,0
4143,a man kills what he loves,0
4144,Why?,0


In [36]:
titles.to_csv('./titles.csv')

## Save Me
#bodies = pd.concat([ven_body, usa_body], axis=0)